In [34]:
import os # file handling
import numpy as np # linear algebra
from tqdm import tqdm # progress bar
from PIL import Image, ImageOps # image processing
from pymongo import MongoClient # MongoDB
from IPython.display import clear_output # clear output

In [35]:
MIRROR = [False, True] # symétrie axiale
INVERSION = [False, True] # inversion des couleurs
ROTATION = [0, 90, 180, 270] # rotation de l'image

In [36]:
# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/Scibio")  # Remplacez par l'URL de votre serveur MongoDB
db = client["Scribio"]  # Nom de la base de données
# collection = db["Dataset_v2"]  # Collection de base
collection_Maj = db["Dataset_Maj_v2"]  # Collection par type
collection_Min = db["Dataset_Min_v2"]  # Collection par type
collection_Nbr = db["Dataset_Nbr_v2"]  # Collection par type

In [37]:
# Transformation des photos en dataframe
def photo_to_dataframe(chemins_photos: str) -> None:
    """Transforme les photos en tableau de pixels et les sauvegarde dans une base de données MongoDB. 
    Les photos sont redimensionnées, retournées, converties en noir et blanc, les couleurs peuvent être inversées 
    et les données sont mises à l'échelle.

    Args:
        chemins_photos (str): Chemin des photos (on veut le dossier contenant les photos)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    tab_img = [] # Initialisation de la liste des images (dictionnaire avec les infos des images)
    type_label = ""
    
    # Split du chemin pour recupérer les informations
    chemin_split = chemins_photos[0].split("\\")
    
    # Affichage des dossiers en cours de traitement 
    print("..\\" + chemin_split[2]) # Dataset
    print("   \\" + chemin_split[3]) # dataset, datasetDigit, datasetSmall
    print("    \\" + chemin_split[4]) # Répartition : test, train, validation
    print("     \\" + chemin_split[5]) # label (26 + 9 + 26)
    
    # Récupération du type de label (Majuscule, Minuscule, Nombre)
    if chemin_split[3] == "dataset":
        type_label = "Maj"
    elif chemin_split[3] == "datasetDigit":
        type_label = "Min"
    elif chemin_split[3] == "datasetSmall":
        type_label = "Nbr"
    
    # Parcours et transformation des photos avec barre de progression 
    with tqdm(total=len(chemins_photos), desc="Proccessing" , position=0, leave=True) as pbar:
        for chemin in chemins_photos: # Parcours des chemins des photos
            
            pbar.set_postfix(img=chemin[-10:]) # Mettre à jour la barre de progression avec des informations supplémentaires
            
            img = Image.open(chemin) # 300 x 300
            
            # Choix de la taille de l'image
            # nouvelle_taille = (50, 50) # 2500
            # nouvelle_taille = (28, 28) # 784
            nouvelle_taille = (25, 25) # 625
            
            img_redimensionnee = img.resize(nouvelle_taille) # Redimensionner l'image
            nb_image = img_redimensionnee.convert('L') # Convertion en noir et blanc
            
            # Transformation de l'image (x16)
            for rotation in ROTATION: # Rotations
                for inversion in INVERSION: # Inversions de couleur
                    for mirror in MIRROR: # Symétrie axiale
                    
                        # Rotation de l'image
                        image_finale = nb_image.rotate(rotation, expand=True)
                        
                        # Inversion de l'image (couleur)
                        if inversion:
                            image_finale = ImageOps.invert(image_finale)
                            
                        if mirror:
                            image_finale = ImageOps.mirror(image_finale)
                        
                        #  Récupération de la liste des pixels
                        tab = []
                        for i in range(image_finale.size[1]):
                            row = []
                            for y in range(image_finale.size[0]):
                                # Récupérer la couleur du pixel
                                row.append(image_finale.getpixel((y, i)))
                            tab.append(row)
                            
                        tab_numpy = np.array(tab) / 255 # mise à l'échelle des données
                        tab_numpy_flatten = tab_numpy.flatten() # Transformation en tableau 1D
                        tab_img.append({  # Ajout du tableau dans la liste
                            "Label": chemin_split[5],
                            "Repartition": chemin_split[4],
                            "Type": type_label,
                            "Rotation": rotation,
                            "Inversion": inversion,
                            "Symetrie": mirror,
                            "Chemin": chemin.replace('\\', '/'),
                            "Tableau_Pixels": tab_numpy_flatten.tolist()
                        })
            
            pbar.update(1) # Mettre à jour la barre de progression
    
    print()
    print(f"Envoie de {len(tab_img):_} lignes dans la base de données ...") # Affichage de la progression
    
    # Insertion des données dans la base de données
    if chemin_split[3] == "dataset":
        collection_Maj.insert_many(tab_img) 
    elif chemin_split[3] == "datasetDigit":
        collection_Min.insert_many(tab_img)
    elif chemin_split[3] == "datasetSmall":
        collection_Nbr.insert_many(tab_img)
    
    pbar.close() # Fermer la barre de progression
    
    clear_output()  # Effacer l'output précédent (sinon pb d'affichage de la barre de progression)

In [38]:
# Parcours des dossiers et sous-dossiers pour recréer le dataset (recursion)
def recreate_dataset(chemin: str) -> None:
    """Réecriture du dataset: transformation des photos en tableau de pixels et les sauvegarde dans un fichier csv
    Cette fonction est récursive et doit être executer sur un dossier contenant la même arborescence que le dossier ../Dataset

    Args:
        chemin (str): chemin du dataset
    """
    dossiers = [f for f in os.listdir(chemin) if os.path.isdir(os.path.join(chemin, f))]
    if len(dossiers) == 0: # Si aucun dossier n'est trouvé, on est dans le cas de base
        fichiers_png = [os.path.join(chemin, f) for f in os.listdir(chemin) if f.endswith('.png')] # on recupère et stocke les fichiers .png
        photo_to_dataframe(fichiers_png) # On transforme les photos en tableau en df qu'on save dans une bdd
    else : # Sinon, on continue de parcourir les dossiers
        for dossier in dossiers:
            recreate_dataset(os.path.join(chemin, dossier))

In [39]:
# Appel de la fonction de parcours des dossiers
recreate_dataset("..\..\Dataset")
# Temps v1 : 145m 20.9s => 2h 25m 20.9s
# Temps v2 : 

..\Dataset
   \dataset
    \test
     \A


Proccessing: 100%|██████████| 1453/1453 [00:08<00:00, 176.75it/s, img=001452.png]



Envoie de 23_248 lignes dans la base de données ...


KeyboardInterrupt: 